In [1]:
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import numpy as np
import time
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import MultiStepLR
import glob 
import os
import cv2
from PIL import Image
import random
from dataset import get_data_loader
from tqdm import tqdm
import time
import cv2
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import albumentations as A
from utils import plot_acc,plot_loss
import shutil
from model import build_model

In [2]:
def train_model(model, criterion, optimizer, num_epochs=3,epoch_start=0,save_img_false=False):
    loss_train = []
    loss_val = []
    acc_train = []
    acc_val = []
    for epoch in range(epoch_start,num_epochs):
        best_acc_val = 0
        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0
            count = 1 
            tepoch =  tqdm(Dataloader[phase])
            for inputs, labels, path in tepoch:
                tepoch.set_description(f"Epoch {epoch}")
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() 
                running_corrects += (torch.sum(preds == labels.data)/ len(inputs)).item() 
                loss_training = running_loss/count
                acc_training = running_corrects/count

                if save_img_false and epoch >= 3:
                    path_save = 'training/epoch_' + str(epoch)
                    isExist = os.path.exists(path_save)
                    if not isExist:
                        os.makedirs(path_save)
                    check_false = preds == labels.data
                    for idx_check,i in enumerate(check_false):
                        if i == False:
                            output_sigmoid = torch.sigmoid(outputs[idx_check])
                            shutil.copy(path[idx_check],path_save + "/" + name_label[labels.data[idx_check]] +
                            "_{:.2f}_{:.2f}_{:.2f}_{:.2f}_".format(
                                *output_sigmoid.cpu().detach().numpy())+ path[idx_check].split('/')[-1])
                if phase == 'train':
                    tepoch.set_postfix(Loss_train=f" {loss_training:.4f}",acc_train=f"{acc_training:.4f}")

                else:
                    tepoch.set_postfix(Loss_val=f" {loss_training:.4f}",acc_val=f"{acc_training:.4f}")

                time.sleep(0.1)
                count +=1 

            if acc_training > best_acc_val and phase == 'validation':
                best_acc_val = acc_training
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    }, 'checkpoint/resnext_best.pt')
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                }, f'checkpoint/resnext_epoch_{epoch}.pt')
            if phase == 'train':
                loss_train.append(round(loss_training,4))
                acc_train.append(round(acc_training,4))
            else:
                loss_val.append(round(loss_training,4))
                acc_val.append(round(acc_training,4))    
        scheduler.step()
        plot_acc(acc_train,acc_val)
        plot_loss(loss_train,loss_val)
    return model

In [2]:
path_data = '/home/a4000/Data/tinhnv/classifier_car/Data_training/Data_training_0801'

name_label =  ['unknown','car','recaptured_images','can_canh']

Dataloader = get_data_loader(path_data,name_label)

416
[INFO]: Number of training examples: 416


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

lr_1 = 0.01
lr_2 = 0.01

num_classes = Dataloader['train'].dataset.classes

f = open("class_names.py", "w")
f.write("class_names = "+str(Dataloader['train'].dataset.name_label))
f.close()

model = build_model(num_classes=num_classes)
model.to(device)

In [ ]:
parameters = []
for name,param in model.named_parameters():
    if 'classifier' in name:
        parameters += [{'params': [param],'lr':lr_2}]
    else:
        parameters += [{'params': [param],'lr':lr_1}]

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(parameters)

# scheduler = MultiStepLR(optimizer, milestones=[10,20], gamma=0.1)
lambda1 = lambda epoch: 0.9 ** epoch
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)

model_trained = train_model(model, criterion, optimizer, num_epochs=50,save_img_false=True)